## **CSV Output Parser**

**Output Parser 없이 리스트 형식 답변 생성하기**

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature = 0)

#ChatPromptTemplate에 SystemMessage로 LLM의 역할과 출력 형식 지정
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="스릴러 영화 3개를 추천해줘.")
answer = llm.invoke(messages)
result = answer.content
print(result)

**CSV 파서로 리스트 형식 답변 생성**

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
#CSV 파서 선언
output_parser = CommaSeparatedListOutputParser()
#CSV 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()
#프롬프트 템플릿의 partial_variables에 CSV 형식 지정 프롬프트 주입
prompt = PromptTemplate(
    template="List {number} {subject}. answer in Korean \n{format_instructions}",
    input_variables=["subject", "number"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

#프롬프트템플릿-모델-Output Parser를 체인으로 연결
chain = prompt | model | output_parser
chain.invoke({"subject": "공포 영화", "number": "4"})

In [ ]:
format_instructions

## **Datetime Parser**

**[Datetime으로만 답변하게 만들기]**

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()

template = """
    Answer the users question:
    {question}

    {format_instructions}
"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser
output = chain.invoke({"question": "비트코인은 언제 개발됐어?"})
print(output)

In [ ]:
output_parser.get_format_instructions()

## **JSON Parser**

In [ ]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define your desired data structure.
class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")

# And a query intented to prompt a language model to populate the data structure.
country_query = "아르헨티나는 어떤 나라야?"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
chain.invoke({"query": country_query})